Simple Neuron implementation from scratch 

In [13]:
import numpy as np

def relu(output):
    return np.maximum(0,output)

weight = np.random.rand()
bias = np.random.randint(1,10)
input = np.random.randint(1,100)

output = np.dot(weight, input) + bias
print(relu(output))

In [21]:
weights = [[.3,.6,.7],
[.2,.54,.87],
[.96,.23,.55]]

inputs = [[1],
[2],
[3]]

bias = [[2],
[3],
[5]]

output = np.dot(weights, inputs) + bias
print(relu(output))

[[5.6 ]
 [6.89]
 [8.07]]


Simple 3 Layered Neural Network (Forward Propagation)

In [19]:
import numpy as np
input_original = np.array([[1],[2],[3],[4],[5]]) #(5x1)

#(5x3) matrix: 5 inputs, 3 neurons
weights_layer1 = np.hstack([
   np.array([[.3],[.2],[.5],[.23],[.6]]), #Neuron 1 Weights
   np.array([[.8],[.232],[.9],[.53],[.66]]), #Neuron 2 Weights
   np.array([[.32],[.27],[.655],[.23],[.46]]) #Neuron 3 Weights
]) 

#(3x2) matrix: 3 inputs, 2 neurons
weights_layer2 = np.hstack([
   np.array([[.3],[.2],[.5]]), #Neuron 1 Weights
   np.array([[.8],[.232],[.9]]), #Neuron 2 Weights
]) 

#(2x1), 2 inputs for 1 neuron
weight_final = np.hstack([
   np.array([[0.63],[.75]]) #Final Neuron Weights
])

bias_layer1 = np.array([[3],[1],[5]]) #(3x1)
bias_layer2 = np.array([[1],[2]]) #(2x1)
bias_final = np.array([[7]])

def hidden_layer1(inputs, weights, bias):
   #(1x5)*(5x3) + (1x3), (inputs^T * weights) + bias^T
   output = np.dot(inputs.T, weights) + bias.T #(1x3)
   return np.maximum(0,output)

def hidden_layer2(inputs, weights, bias):
   #(1x3)*(3x2) + (1x3), (inputs^T * weights) + bias^T
   output = np.dot(inputs, weights) + bias.T #(1x3)
   return np.maximum(0,output)

def final_layer(inputs, weights, bias):
   #(2x1)*(1x1) + (1x1)
   output = np.dot(inputs, weights) + bias
   return np.maximum(0,output)
print("Inputs:\n",input_original)
print("Inputs for 1st layer:\n",hidden_layer1(input_original, weights_layer1, bias_layer1))
inputs_2 = np.array(hidden_layer1(input_original, weights_layer1, bias_layer1))
print("Inputs for 2nd layer:\n",hidden_layer2(inputs_2, weights_layer2, bias_layer2))
inputs_final = np.array(hidden_layer2(inputs_2, weights_layer2, bias_layer2)) 
output_final = final_layer(inputs_final, weight_final, bias_final)
print("Final output:\n",output_final.squeeze())

Inputs:
 [[1]
 [2]
 [3]
 [4]
 [5]]
Inputs for 1st layer:
 [[ 9.12  10.384 11.045]]
Inputs for 2nd layer:
 [[11.3353   21.645588]]
Final output:
 30.37543


3 Layered Back-Propagation Neural Network

In [8]:
import numpy as np


#        ================== Forward Pass ================== 
''' 
Feed Forward Multi Layer Neural Network:
1. y = mx + c | no = w x i + b | neuron_output = weights x inputs + bias
2. activation_function(neuron_output)
'''

#(3x1)
inputs_original = np.array([[1],[2],[3]]) 

#(3x3)
weights_layer1 = np.hstack([
   np.array([[.3],[.34],[.67]]), # Weights for Neuron 1, (3x1)
   np.array([[.2],[.23],[.21]]), # Weights for Neuron 2, (3x1)
   np.array([[.4],[.43],[.11]]) # Weights for Neuron 3, (3x1)
])

#(3x2)
weights_layer2 = np.hstack([
   np.array([[.3],[.34],[.44]]), # Weights for Neuron 1, (3x1)
   np.array([[.2],[.23],[.75]]), # Weights for Neuron 2, (3x1)
])

#(2x1)
weight_final = np.array([[.33],[.66]]) # Weight for final Neuron

bias_layer1 = np.array([[1],[2],[1]]) # Biases for Hidden 1st Layer, (3x1)
bias_layer2 = np.array([[2],[3]]) # Biases for Hidden 2nd Layer, (2x1)
bias_final = np.array([1.1]) # Bias for Final Layer, (1x1)

def hidden_layer1(inputs, weights, bias):
   # (3x1)*(3x3) + (1x3)
   output = np.dot(inputs.T, weights) + bias.T
   return np.maximum(0, output)

def hidden_layer2(inputs, weights, bias):
   # (1x3)*(3x2) + (1x2)
   output = np.dot(inputs, weights) + bias.T
   return np.maximum(0, output)

def final_layer(inputs, weights, bias):
   # (1x2)*(2x1) + (1x1)
   output = np.dot(inputs, weights) + bias
   return np.maximum(0, output)

output_layer_1 = hidden_layer1(inputs_original, weights_layer1, bias_layer1)
output_layer_2 = hidden_layer2(output_layer_1, weights_layer2, bias_layer2)
final_layer = final_layer(output_layer_2, weight_final, bias_final)
print("================== Forward Pass ==================")
print("1st Layer output:\n", output_layer_1,"\n")
print("2st Layer output:\n", output_layer_2,"\n")
print("Final Layer output:\n", final_layer.squeeze().round(3),'\n')


print("================== Backward Pass ==================")  
'''

==================================== Back Propagation =====================================
1. Loss Function (Mean Squared Error):
   l = 1/2(a - p)^2 | loss = 0.5(actual - predicted)^2

2. Gradients:
Note: Weights, Biases, Final Output are derivated/unpacked to update wrt to gradient/changes/difference between actual & predicted.

   [Final/Output Layer]
      Loss w.r.t Output: (Loss Function Derivative)
         dL/dp = (p - a) | loss_output = predicted - actual
      Predicted w.r.t Weight:
         dp/dw = A | predicted_weight = neuron_output
      Loss w.r.t Weight:
         dL/dw = A x (dL/dp) | loss_output_weight = neuron_output x loss_output
      Loss w.r.t Bias:
         dL/db = sum(dL/dp) | loss_bias = sum(loss_output)

   [Hidden Layer]
      Loss w.r.t  Pre-Activation Neuron Output (Z):
         dL/dZ = dL/dA x dA/dZ = (dL/dp x w).ReLu(Z) | loss_raw_neuron_output = (loss_output x weight).activation_function(raw_neuron_output)
      Loss w.r.t Weight:
         dL/dw = A x (dL/dZ) | loss_weight = neuron_output x (loss_raw_neuron_output)
      Loss w.r.t Bias:
         dL/db = sum(dL/dZ) | loss_bias = sum(loss_raw_neuron_output)
   
   [Input Layer]
      Loss w.r.t Pre-Activation Neuron Output (Z):
         dL/dZ = (dL/dZ x w).ReLu(Z) | loss_raw_neuron_output = (loss_raw_neuron_output x weight).activation_function(raw_neuron_output)
      Loss w.r.t Weight:
         dL/dw = i x dL/dZ | loss_weight = input x loss_raw_neuron_output
      Loss w.r.t Bias:
         dL/db = sum(dL/dZ) | loss_bias = sum(loss_raw_neuron_output)

   [Update Weights]
      w = w - (lr x dL/dw) | weight = weight - learning_rate x loss_weight
'''

error = 1 - final_layer
print("Error(Actual-Predicted): ",error)

================== Forward Pass ==================
1st Layer output:
 [[3.99 3.29 2.59]] 

2st Layer output:
 [[5.4552 6.4972]] 

Final Layer output:
 7.188 

================== Backward Pass ==================
Error(Actual-Predicted):  [[-6.188368]]
